In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import cv2
import os
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader,Dataset
import albumentations as albu
from skimage.color import gray2rgb
import functools
import torch
from tqdm.auto import tqdm

In [2]:
from efficientnet_pytorch import EfficientNet
from source.model.ResNet3D import ResNet, resnet18
import source.model.ShuffleNet3D as shufflenet

In [3]:
import os
class data_config:
    train_csv_path = 'D:/dataset/train.csv'
    jpeg_dir ='../input/rsna-str-pe-detection-jpeg-256/train-jpegs'
    ids = ['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']    
    label_lstm = ['pe_present_on_image','negative_exam_for_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe','rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate']
        
class efficientnetb0:
    model_name="efficientnet-b0"
    batch_size = 1
    WORKERS = 0
    classes =9
    epochs = 1
    optimizer = "torch.optim.AdamW"
    optimizer_parm = {'lr':1e-3,'weight_decay':0.00001}
    scheduler = "torch.optim.lr_scheduler.CosineAnnealingLR"
    scheduler_parm = {'T_max':5500,'eta_min':0.000001}
    loss_fn = 'torch.nn.BCEWithLogitsLoss'
    MODEL_PATH = 'log/cpt'
    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)
        
class opt:
    batch_size = 16
    n_classes = 9
    resnet_shortcut = 'A'
    sample_size = 112 # ?
    sample_duration = 16
    cnn_pretrained = 'weights/kinetics_shufflenet_2.0x_G3_RGB_16_best.pth'
    arch = 'shufflenet'
    width_mult = 2
    groups = 3

In [4]:
train_csv_path = 'D:/dataset/train.csv'
#jpeg_dir = '../input/rsna-str-pe-detection-jpeg-256/train-jpegs'

In [5]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
# def get_fold(train,FOLD_NUM = 5):
#     train_image_num_per_patient = train.groupby('StudyInstanceUID')['SOPInstanceUID'].nunique()
#     target_cols = [c for i, c in enumerate(train.columns) if i > 2]
    
#     train_per_patient_char = pd.DataFrame(index=train_image_num_per_patient.index, columns=['image_per_patient'], data=train_image_num_per_patient.values.copy())
#     for t in target_cols:
#         train_per_patient_char[t] = train_per_patient_char.index.map(train.groupby('StudyInstanceUID')[t].mean())
        
    
#     bin_counts = [40] #, 20]
#     digitize_cols = ['image_per_patient'] #, 'pe_present_on_image']
#     non_digitize_cols = [c for c in train_per_patient_char.columns if c not in digitize_cols]
#     for i, c in enumerate(digitize_cols):
#         bin_count = bin_counts[i]
#         percentiles = np.percentile(train_per_patient_char[c], q=np.arange(bin_count)/bin_count*100.)
#         train_per_patient_char[c+'_digitize'] = np.digitize(train_per_patient_char[c], percentiles, right=False)
        
#     train_per_patient_char['key'] = train_per_patient_char[digitize_cols[0]+'_digitize'].apply(str)
#     for c in digitize_cols[1:]:
#         train_per_patient_char['key'] = train_per_patient_char['key']+'_'+train_per_patient_char[c+'_digitize'].apply(str)
#     folds = FOLD_NUM
#     kfolder = StratifiedKFold(n_splits=folds, shuffle=True, random_state=719)
#     val_indices = [val_indices for _, val_indices in kfolder.split(train_per_patient_char['key'], train_per_patient_char['key'])]
#     train_per_patient_char['fold'] = -1
#     for i, vi in enumerate(val_indices):
#         patients = train_per_patient_char.index[vi]
#         train_per_patient_char.loc[patients, 'fold'] = i
#     return train_per_patient_char

# def split_train_val_lstm(data_config,fold,FOLD_NUM=5):
    
#     main_df = pd.read_csv(data_config.train_csv_path)
#     train_df = main_df[data_config.ids+data_config.label_lstm]
#     train_per_patient_char = get_fold(main_df,FOLD_NUM)
#     TID = train_per_patient_char[train_per_patient_char.fold!=fold].index
#     VID = train_per_patient_char[train_per_patient_char.fold==fold].index
#     t_df = train_df[train_df['StudyInstanceUID'].isin(TID)]
#     v_df = train_df[train_df['StudyInstanceUID'].isin(VID)]
#     return t_df,v_df
    

In [6]:
#t_df,v_df = split_train_val_lstm(data_config,fold=0,FOLD_NUM=5)

#folds = StratifiedKFold(n_splits=CFG['fold_num'], shuffle=True, random_state=CFG['seed']).split(np.arange(train.shape[0]), train.label.values)

In [7]:
# path256 = f"{data_config.jpeg_dir}/*/*/*.jpg"
# data = glob.glob(path256)
# new_df = []
# for row in tqdm(data):
#     StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID = row.split("/")[-3:]
#     num,SOPInstanceUID = SOPInstanceUID.replace(".jpg","").split("_")
#     new_df.append([StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,num])
# s_df = pd.DataFrame(new_df)
# s_df.columns = list(t_df.columns[:3])+["slice"]
# t_df = t_df.merge(s_df,on=list(t_df.columns[:3]),how='left')
# v_df = v_df.merge(s_df,on=list(v_df.columns[:3]),how='left')
    

In [8]:
# t = t_df.groupby(list(t_df.columns[:2]))
# mini_dfs= []
# for i,row in tqdm(t_df.groupby(list(t_df.columns[:2]))):
#     if len(row)>400:
#         continue
#     mini_dfs.append(row.sort_values("slice"))
# mini_dfs_val = []
# for i,row in tqdm(v_df.groupby(list(v_df.columns[:2]))):
#     if len(row)>400:
#         continue
#     mini_dfs_val.append(row.sort_values("slice"))

In [9]:
# class CTDataset3D(Dataset):
#     def __init__(self,df,jpeg_dir,transforms = None,preprocessing=None,size=256,mode='val'):
#         self.df_main = df
#         self.jpeg_dir = jpeg_dir
#         self.transforms = transforms
#         self.preprocessing = preprocessing
#         self.size=size

#     def __getitem__(self, idx):
#         mini = self.df_main[idx].values
#         all_paths = [f"{self.jpeg_dir}/{row[0]}/{row[1]}/{row[-1]}_{row[2]}.jpg" for row in mini]
#         img = [self.transforms(image=cv2.imread(p))['image'] for p in all_paths]
#         label = mini[:,3:-1].astype(int)
#         if self.preprocessing:
#             img = [self.preprocessing(image=im)['image'] for im in img]
#         return np.array(img),torch.from_numpy(label[:,0]),torch.from_numpy(label[0,1:])

#     def __len__(self):
#         return len(self.df_main)

In [10]:
def get_training_augmentation(y=256, x=256):
    train_transform = [albu.Resize(y, x)]
    return albu.Compose(train_transform)


formatted_settings = {
    "input_size": [3, 224, 224],
    "input_range": [0, 1],
    "mean": [0.485, 0.456, 0.406],
    "std": [0.229, 0.224, 0.225],
}


def preprocess_input(
    x, mean=None, std=None, input_space="RGB", input_range=None, **kwargs
):

    if input_space == "BGR":
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 255.0

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x


def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)


def get_validation_augmentation(y=256, x=256):
    """Add paddings to make image shape divisible by 32"""
    test_transform = [albu.Resize(y, x)]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    """
    Convert image or mask.
    """
    return x.transpose(2, 0, 1).astype("float32")


def norm(img):
    img -= img.min()
    return img / img.max()

In [14]:
# preprocessing_fn = functools.partial(preprocess_input, **formatted_settings)
# train_dataset = CTDataset3D(mini_dfs,data_config.jpeg_dir,
#                             transforms=get_training_augmentation(),preprocessing=get_preprocessing(preprocessing_fn))
# val_dataset = CTDataset3D(mini_dfs_val,data_config.jpeg_dir,
#                             transforms=get_validation_augmentation(),preprocessing=get_preprocessing(preprocessing_fn))

from source.dataset import VideoDataset

clip_len = 16
train_dataset = VideoDataset(root_dir = "./dataset", dataset='kids', split='train', clip_len=clip_len)
val_dataset   = VideoDataset(root_dir = "./dataset", dataset='kids', split='val', clip_len=clip_len)


Number of train videos: 2663
Number of val videos: 670


In [15]:
# x,y,y1 = train_dataset[0]
# x.shape,y.shape,y1.shape

In [16]:
model_config = efficientnetb0

In [17]:
# import torch
# from efficientnet_pytorch import EfficientNet
# cnn = EfficientNet.from_pretrained('efficientnet-b0',num_classes=9).cuda()
# cnn = torch.nn.DataParallel(cnn)

In [18]:
# cnn = resnet18(
#                 num_classes=opt.n_classes,
#                 shortcut_type=opt.resnet_shortcut,
#                 sample_size=opt.sample_size,
#                 sample_duration=opt.sample_duration)
cnn = shufflenet.get_model(
            groups=opt.groups,
            width_mult=opt.width_mult,
            num_classes=opt.n_classes)
cnn = torch.nn.DataParallel(cnn)

/usr/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [19]:
pretrained_dict = torch.load(opt.cnn_pretrained, map_location=torch.device('cpu'))
assert opt.arch == pretrained_dict['arch']
cnn_dict = cnn.state_dict() # 현재 신경망 상태 로드

# print("===========================")
# print("len(cnn_dict) : ", len(cnn_dict.keys()))
# print(cnn_dict.keys())
# print("before len(pretrained_dict) : ", len(pretrained_dict['state_dict'].keys()))
# print(pretrained_dict['state_dict'].keys())
# print("===========================")

pretrained_dict = { k:v for k,v in pretrained_dict['state_dict'].items() if k in cnn_dict and v.size() == cnn_dict[k].size() }
# print("after len(pretrained_dict) : ", len(pretrained_dict.keys()))
# print(pretrained_dict.keys())
# print("===========================")


for ((k1,v1),(k2,v2)) in zip(pretrained_dict.items(), cnn_dict.items()):
    if k1 not in cnn_dict or v1.size() != cnn_dict[k1].size() :
        print("cannot load layer")
        print("k1 : ", k1)
        print("k2 : ", k2)
        print("v1.size() : ", v1.size())
        print("cnn_dict[k1].size() : ", cnn_dict[k1].size())
print("================================================")
cnn_dict.update(pretrained_dict)
cnn.load_state_dict(cnn_dict)

FileNotFoundError: [Errno 2] No such file or directory: 'weights/kinetics_shufflenet_2.0x_G3_RGB_16_best.pth'

In [ ]:
from torch import nn
from torch.nn import functional as F

class NeuralNet(nn.Module):
    def __init__(self,cnn, embed_size=1280, LSTM_UNITS=64, DO = 0.3):
        super(NeuralNet, self).__init__()
        self.cnn = cnn.module
        self.cnn.eval().cuda()
        self.avgpool = torch.nn.AdaptiveAvgPool2d(1)
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear_pe = nn.Linear(LSTM_UNITS*2, 1)
        self.linear_global = nn.Linear(LSTM_UNITS*2, 9)

    def forward(self, x, lengths=None):
        with torch.no_grad():
            embedding = self.cnn(x)
            embedding = self.avgpool(embedding)
            b,f,_,_ = embedding.shape
            embedding = embedding.reshape(1,b,f)
        self.lstm1.flatten_parameters()
        h_lstm1, _ = self.lstm1(embedding)
        self.lstm2.flatten_parameters()
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2

        output = self.linear_pe(hidden)
        output_global = self.linear_global(hidden.mean(1))
        return output,output_global

In [ ]:
model = NeuralNet(cnn).cuda()

In [ ]:
optimizer = eval(model_config.optimizer)(model.parameters(),**model_config.optimizer_parm)
scheduler = eval(model_config.scheduler)(optimizer,**model_config.scheduler_parm)
loss_fn = eval(model_config.loss_fn)()#pos_weight=torch.FloatTensor(model_config.pos_weight).cuda())

In [ ]:
import torch
import numpy as np
from tqdm.auto import tqdm
import os
class trainer:
    def __init__(self,loss_fn,model,optimizer,scheduler,config):
        self.loss_fn = loss_fn
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.config = config

        
    def batch_train(self, batch_imgs, batch_labels, batch_idx):
        batch_imgs, batch_labels = batch_imgs.cuda().float(), batch_labels.cuda().float()
        #print("[batch_train]batch_imgs.shape : ", batch_imgs.shape)
        predicted = self.model(batch_imgs)
        loss = self.loss_fn(predicted[0].float().reshape(-1), batch_labels.reshape(-1))

        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.item(), predicted
    
    def batch_valid(self, batch_imgs,get_fet):
        self.model.eval()
        batch_imgs = batch_imgs.cuda()
        with torch.no_grad():
            predicted = self.model(batch_imgs)
            predicted[0] = torch.sigmoid(predicted[0])
            predicted[1] = torch.sigmoid(predicted[1])
        return predicted
    
    def train_epoch(self, loader):
        self.model.train()
        tqdm_loader = tqdm(loader)
        current_loss_mean = 0
        for batch_idx, (imgs,labels) in enumerate(tqdm_loader):
            #print("[train_epoch]imgs.shape : ", imgs.shape)
            loss, predicted = self.batch_train(imgs, labels, batch_idx)
            current_loss_mean = (current_loss_mean * batch_idx + loss) / (batch_idx + 1)
            tqdm_loader.set_description('loss: {:.4} lr:{:.6}'.format(
                    current_loss_mean, self.optimizer.param_groups[0]['lr']))
            self.scheduler.step(batch_idx)
            if batch_idx>10:
                break
        return current_loss_mean
    
    def valid_epoch(self, loader,name="valid"):
        self.model.eval()
        tqdm_loader = tqdm(loader)
        current_loss_mean = 0
        correct = 0
        for batch_idx, (imgs,labels) in enumerate(tqdm_loader):
            with torch.no_grad():
                batch_imgs = imgs.cuda().float()
                batch_labels = labels.cuda().float()

                predicted = self.model(batch_imgs)
                loss = self.loss_fn(predicted[1].float().reshape(-1),batch_labels.float().reshape(-1)).item()

                current_loss_mean = (current_loss_mean * batch_idx + loss) / (batch_idx + 1)
                tqdm_loader.set_description(f"loss : {current_loss_mean:.4}")
            if batch_idx>10:
                break
        score = 1-current_loss_mean
        print('metric {}'.format(score))
        return score
    
    def run(self,train_loder,val_loder):
        best_score = -100000
        for e in range(self.config.epochs):
            print("----------Epoch {}-----------".format(e))
            current_loss_mean = self.train_epoch(train_loder)
            score = self.valid_epoch(val_loder)
            if best_score < score:
                best_score = score
                torch.save(self.model.state_dict(),self.config.MODEL_PATH+"/{}_best.pth".format(self.config.model_name))

    def batch_valid_tta(self, batch_imgs):
        batch_imgs = batch_imgs.cuda()
        predicted = model(batch_imgs)
        tta_flip = [[-1],[-2]]
        for axis in tta_flip:
            predicted += torch.flip(model(torch.flip(batch_imgs, axis)), axis)
        predicted = predicted/(1+len(tta_flip))
        predicted = torch.sigmoid(predicted)
        return predicted.cpu().numpy()
            
    def load_best_model(self):
        if os.path.exists(self.config.MODEL_PATH+"/{}_best.pth".format(self.config.model_name)):
            self.model.load_state_dict(torch.load(self.config.MODEL_PATH+"/{}_best.pth".format(self.config.model_name)))
            print("load best model")
        
    def predict(self,imgs_tensor,get_fet = False):
        self.model.eval()
        with torch.no_grad():
            return self.batch_valid(imgs_tensor,get_fet=get_fet)

In [ ]:
Trainer = trainer(loss_fn,model,optimizer,scheduler,config=model_config)

In [ ]:
train = DataLoader(train_dataset, batch_size=opt.batch_size, shuffle=True, num_workers=model_config.WORKERS, pin_memory=False)
val = DataLoader(val_dataset, batch_size=opt.batch_size, shuffle=False, num_workers=model_config.WORKERS, pin_memory=False)

In [ ]:
Trainer.run(train,val)